<a href="https://colab.research.google.com/github/katrin2202/text-summarization-in-nlp/blob/main/BART.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets evaluate rouge_score

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
from datasets import load_dataset

dataset = load_dataset("Katerina-gopher/ru-summary")

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['summ', 'text'],
        num_rows: 121833
    })
    test: Dataset({
        features: ['summ', 'text'],
        num_rows: 17512
    })
    validation: Dataset({
        features: ['summ', 'text'],
        num_rows: 12103
    })
})

In [ ]:
from transformers import AutoTokenizer

checkpoint = "facebook/bart-large-xsum"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [ ]:
prefix = "summarize: "


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["summ"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/121833 [00:00<?, ? examples/s]

Map:   0%|          | 0/17512 [00:00<?, ? examples/s]

Map:   0%|          | 0/12103 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model="Katerina-gopher/bart-ru-summary")
# data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model="facebook/bart-large-xsum")

In [ ]:
import evaluate

rouge = evaluate.load("rouge")

In [ ]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
!git lfs install
!git clone https://huggingface.co/Katerina-gopher/bart-ru-summary

Error: Failed to call git rev-parse --git-dir: exit status 128 
Git LFS initialized.
Cloning into 't5-ru-summary'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 20 (delta 4), reused 0 (delta 0), pack-reused 4
Unpacking objects: 100% (20/20), 3.25 KiB | 665.00 KiB/s, done.


In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained("bart-ru-summary")
# model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-xsum")

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="bart-ru-summary",
    # output_dir="facebook/bart-large-xsum",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=15,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    # train_dataset=tokenized_dataset["train"],
    # eval_dataset=tokenized_dataset["test"],
    train_dataset=tokenized_dataset["test"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/content/t5-ru-summary is already a clone of https://huggingface.co/Katerina-gopher/t5-ru-summary. Make sure you pull the latest changes with `repo.git_pull()`.
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,2.177200,2.045876,0.000000,0.000000,0.000000,0.000000,19.000000
2,2.159000,2.029339,0.000000,0.000000,0.000000,0.000000,19.000000
3,2.148800,2.016506,0.000000,0.000000,0.000000,0.000000,19.000000


Several commits (2) will be pushed upstream.
Several commits (3) will be pushed upstream.
Several commits (4) will be pushed upstream.
Several commits (5) will be pushed upstream.
Several commits (6) will be pushed upstream.


KeyboardInterrupt: ignored